In [1]:
import modal
from modal import wsgi_app

app = modal.App("tensorboard")
image = modal.Image.debian_slim(python_version="3.11").pip_install(
    "tensorboard",
    "pandas",
    "numpy",
    "Pillow",
    "matplotlib",
    "scikit-learn",
)

vol = modal.Volume.from_name("climate-forecast")

In [2]:
@app.function(
    image=image,
    volumes={"/vol": vol},
    timeout=(60 * 60 * 24),
)
@wsgi_app()
def tensorboard_app():
    import tensorboard

    board = tensorboard.program.TensorBoard()
    board.configure(logdir="/vol/flax/logs/")
    (data_provider, deprecated_multiplexer) = board._make_data_provider()
    wsgi_app = tensorboard.backend.application.TensorBoardWSGIApp(
        board.flags,
        board.plugin_loaders,
        data_provider,
        board.assets_zip_provider,
        deprecated_multiplexer,
    )  # Note: prior to April 2024, "app" was called "stub"
    return wsgi_app

In [ ]:
with app.run():
  import time
  tensorboard_app.local()
  try:
      while True:
          time.sleep(1)
  except KeyboardInterrupt:
      print("Terminating app")